# **Práctica 3**: GANs
# Parte 2: GANs simples

### ALUMNO: JOSÉ JAVIER GUTIÉRREZ GIL

In [ ]:
#!pip install ai_benchmark

In [ ]:
#from ai_benchmark import AIBenchmark
#benchmark_gpu = AIBenchmark(use_CPU=False)
#benchmark_gpu.run_training()

In [ ]:
import tensorflow as tf
if tf.config.list_physical_devices('GPU'): 
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print('(*) gpus List: ',gpus)
    device_name = tf.test.gpu_device_name()
else:
    device_name = '/CPU:0'
print('device_name:',device_name)


# Limitación la memoria de la GPU
#config = tf.compat.v1.ConfigProto (allow_soft_placement=True)
#config.gpu_options.per_process_gpu_memory_fraction = 0.6
#tf.compat.v1.keras.backend.set_session (tf.compat.v1.Session(config=config))

# Permitir crecimiento de la memoria
 
# try:

    # if gpus:
      # Establecer el crecimiento de memoria en True para permitir asignación dinámica de memoria
      # for gpu in gpus:
        # print (' -:',gpu)
        # tf.config.experimental.set_memory_growth(gpu, True)
        # Inicializa el dispositivo de la GPU
        # tf.config.experimental.set_logical_device_configuration(gpu, [tf.config.LogicalDeviceConfiguration(memory_limit = 2000)])

# except Exception as e:
    # Captura la excepción y muestra información sobre el error
    # print("Se produjo una excepción:", e)
   #  print('(*) Invalid device or cannot modify virtual devices once initialized.')


In [ ]:
import pkg_resources
libs = ["protobuf","tfds-nightly","googleapis-common-protos", "mediapipe",
        "tb-nightly", "tensorboard", "tensorflow", "tensorflow-gpu","tensorflow-intel", "tensorflow-metadata", 
        "tf_nightly_intel"]

[lb+' = '+pkg_resources.get_distribution(lb).version for lb in libs] 

### Ejercicio prelab:
Corre el código de pix2pix de la página de tensorflow:
https://www.tensorflow.org/tutorials/generative/pix2pix

### · Ejercicio 1:
Corre el código de pix2pix de la página de tensorflow (el anterior) para el dataset "maps". Testealo pasándole como entrada una imagen del mapa de google maps de la ETSE.

- Se ha descargado el notebook pix2pix_facades.ipynb y ejecutado para estudiar el código. 
- Posteriormente se ha introduido el codigo y modificado en este notebook para que se ejecute con el dataset "map". Se guarda en el notebook **'pix2pix_map.ipynb'**. Las modificaciones realizadas han sido: 

    * Modificamps la variable que guarda el nombre del dataset 'dataset_name = "maps"'  y se descarga el tar.gz de su url '_URL = f'http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/{dataset_name}.tar.gz''
    * El dataset  facades tenía imagenes de 256x512 ( 2 imagenes de 256x256). En este caso, el dataset map tiene dos imagenes de 600x600. Por lo que modificamos el código para que pueda trabajar con estos tamaños.
        * Hacemos quun resize a 6030x3630 y luego un crop aleatorio para eliminar de forma aleatoria pixels tal que se queden en imagenes de 256x256. 
        
De esta forma se puede ejecutar el código del modelo sin ninguna modificación más. Esto se debe hacer ya que el algoritmo pix2pix obliga a tener el mismo tamaño de imagen de entrada y salida. Otra opción sería utilizar el algoritmo cycleGan en el cual no es necesario que se tenga el mismo tamaño de imagen.

Nota: El algoritmo pix2pix utlita una arquitectura U-NET para el generado el cual tiene un codificador decodificador. La arquitectura U-Net fue presentada por Olaf Ronneberger, Philipp Fischer, Thomas Brox en 2015 para la detección de tumores, pero desde entonces se ha descubierto que es útil en múltiples industrias. Como herramienta de segmentación de imágenes, el modelo tiene como objetivo clasificar cada píxel como una de las clases de salida. 

Y el algoritmo PatchGAN  para el decodificador, el cual es una variante del discriminador utilizado en GAN. A diferencia del discriminador convencional, que evalúa la imagen completa, PatchGAN evalúa la imagen en pequeñas regiones llamadas parches. Esto permite que el discriminador tenga más información detallada sobre la imagen y pueda capturar características finas y de alta frecuencia.

En resumen, GAN es un marco de red neuronal generativa que consta de dos partes, mientras que PatchGAN es una variante del discriminador utilizado en GAN que evalúa la imagen en parches en lugar de la imagen completa.
   

## Código pix2pix_maps

In [ ]:
import tensorflow as tf

import os
import pathlib
import time
import datetime

from matplotlib import pyplot as plt
from IPython import display

In [ ]:
dataset_name = "maps" #@param ["cityscapes", "edges2handbags", "edges2shoes", "facades", "maps", "night2day"]

In [ ]:
_URL = f'http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/{dataset_name}.tar.gz'

path_to_zip = tf.keras.utils.get_file(
    fname=f"{dataset_name}.tar.gz",
    origin=_URL,
    extract=True)

path_to_zip  = pathlib.Path(path_to_zip)

PATH = path_to_zip.parent/dataset_name



In [ ]:
list(PATH.parent.iterdir())

In [ ]:
sample_image = tf.io.read_file(str(PATH / 'train/1.jpg'))
sample_image = tf.io.decode_jpeg(sample_image)
print(sample_image.shape)
plt.figure()
plt.imshow(sample_image)

In [ ]:
def load(image_file):
    # Read and decode an image file to a uint8 tensor
    image = tf.io.read_file(image_file)
    image = tf.io.decode_jpeg(image)

    # Split each image tensor into two tensors:
    # - one with a real building facade image
    # - one with an architecture label image 
    w = tf.shape(image)[1]
    w = w // 2
    input_image = image[:, w:, :]
    real_image  = image[:, :w, :]

    # Convert both images to float32 tensors
    input_image = tf.cast(input_image, tf.float32)
    real_image  = tf.cast(real_image, tf.float32)

    return input_image, real_image

inp, re = load(str(PATH / 'train/100.jpg'))
# Casting to int for matplotlib to display the images
plt.figure()
plt.imshow(inp / 255.0)
plt.figure()
plt.imshow(re / 255.0)

In [ ]:
# The facade training set consist of 400 images
BUFFER_SIZE = 400
# The batch size of 1 produced better results for the U-Net in the original pix2pix experiment
BATCH_SIZE  = 1
# Each image is 256x256 in size
IMG_WIDTH   = 256
IMG_HEIGHT  = 256

# resize
RE_IMG_WIDTH   = 630 # --123--
RE_IMG_HEIGHT  = 630 # --123--

In [ ]:
def resize(input_image, real_image, height, width):
    input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image = tf.image.resize(real_image, [height, width],
                               method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image, real_image

In [ ]:
def random_crop(input_image, real_image):
    stacked_image = tf.stack([input_image, real_image], axis=0)
    cropped_image = tf.image.random_crop(
                                  stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image[0], cropped_image[1]

# Normalizing the images to [-1, 1]
 


In [ ]:
# Normalizing the images to [-1, 1]
def normalize(input_image, real_image):
    input_image = (input_image / 127.5) - 1
    real_image = (real_image / 127.5) - 1

    return input_image, real_image

In [ ]:
@tf.function()
def random_jitter(input_image, real_image):
    # Resizing to 286x286
    input_image, real_image = resize(input_image, real_image, RE_IMG_WIDTH, RE_IMG_HEIGHT)

    # Random cropping back to 256x256
    input_image, real_image = random_crop(input_image, real_image)

    if tf.random.uniform(()) > 0.5:
        # Random mirroring
        input_image = tf.image.flip_left_right(input_image)
        real_image = tf.image.flip_left_right(real_image)

    return input_image, real_image

plt.figure(figsize=(6, 6))
for i in range(4):
    rj_inp, rj_re = random_jitter(inp, re)
    plt.subplot(2, 2, i + 1)
    plt.imshow(rj_inp / 255.0)
    plt.axis('off')
plt.show()


for i in range(4):
    rj_inp, rj_re = random_jitter(inp, re)
    plt.subplot(2, 2, i + 1)
    plt.imshow(rj_re / 255.0)
    plt.axis('off')
plt.show()

In [ ]:
def load_image_train(image_file):
    input_image, real_image = load(image_file)
    input_image, real_image = random_jitter(input_image, real_image)
    input_image, real_image = normalize(input_image, real_image)

    return input_image, real_image

def load_image_test(image_file):
    input_image, real_image = load(image_file)
    input_image, real_image = resize(input_image, real_image,
                                   IMG_HEIGHT, IMG_WIDTH)
    input_image, real_image = normalize(input_image, real_image)

    return input_image, real_image 

In [ ]:
train_dataset = tf.data.Dataset.list_files(str(PATH / 'train/*.jpg'))
train_dataset = train_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)


try:
    test_dataset = tf.data.Dataset.list_files(str(PATH / 'test/*.jpg'))
except tf.errors.InvalidArgumentError:
    test_dataset = tf.data.Dataset.list_files(str(PATH / 'val/*.jpg'))
test_dataset = test_dataset.map(load_image_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
OUTPUT_CHANNELS = 3

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
          tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                                 kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(tf.keras.layers.LeakyReLU())

    return result

down_model = downsample(3, 4)
down_result = down_model(tf.expand_dims(inp, 0))
print (down_result.shape)

In [ ]:
def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
                tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                                    padding='same',
                                                    kernel_initializer=initializer,
                                                    use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

up_model  = upsample(3, 4)
up_result = up_model(down_result)
print (up_result.shape) 

In [ ]:
def Generator():
    inputs = tf.keras.layers.Input(shape=[256, 256, 3])

    down_stack = [
        downsample(64, 4, apply_batchnorm=False),  # (batch_size, 128, 128, 64)
        downsample(128, 4),  # (batch_size, 64, 64, 128)
        downsample(256, 4),  # (batch_size, 32, 32, 256)
        downsample(512, 4),  # (batch_size, 16, 16, 512)
        downsample(512, 4),  # (batch_size, 8, 8, 512)
        downsample(512, 4),  # (batch_size, 4, 4, 512)
        downsample(512, 4),  # (batch_size, 2, 2, 512)
        downsample(512, 4),  # (batch_size, 1, 1, 512)
    ]

    up_stack = [
        upsample(512, 4, apply_dropout=True),  # (batch_size, 2, 2, 1024)
        upsample(512, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
        upsample(512, 4, apply_dropout=True),  # (batch_size, 8, 8, 1024)
        upsample(512, 4),  # (batch_size, 16, 16, 1024)
        upsample(256, 4),  # (batch_size, 32, 32, 512)
        upsample(128, 4),  # (batch_size, 64, 64, 256)
        upsample(64, 4),  # (batch_size, 128, 128, 128)
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')  # (batch_size, 256, 256, 3)

    x = inputs

    # Downsampling through the model
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

generator = Generator()
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

In [ ]:
rj_inp, rj_re = random_jitter(inp, re)  # --123--  
print(rj_inp.shape)
gen_output = generator(rj_inp[tf.newaxis, ...], training=False) # --123--  
plt.imshow(gen_output[0, ...])

In [ ]:
LAMBDA = 100

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

    # Mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

    total_gen_loss = gan_loss + (LAMBDA * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
    tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

    x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 256, 256, channels*2)

    down1 = downsample(64, 4, False)(x)  # (batch_size, 128, 128, 64)
    down2 = downsample(128, 4)(down1)  # (batch_size, 64, 64, 128)
    down3 = downsample(256, 4)(down2)  # (batch_size, 32, 32, 256)

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (batch_size, 34, 34, 256)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)  # (batch_size, 31, 31, 512)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 33, 33, 512)

    last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2)  # (batch_size, 30, 30, 1)

    return tf.keras.Model(inputs=[inp, tar], outputs=last)

discriminator = Discriminator()
tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
rj_inp, rj_re = random_jitter(inp, re)  # --123--   
disc_out = discriminator([rj_inp[tf.newaxis, ...], gen_output], training=False)
plt.imshow(disc_out[0, ..., -1], vmin=-20, vmax=20, cmap='RdBu_r')
plt.colorbar()

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

    total_disc_loss = real_loss + generated_loss

    return total_disc_loss

In [ ]:
generator_optimizer     = tf.keras.optimizers.Adam (2e-4, beta_1 = 0.5)
discriminator_optimizer = tf.keras.optimizers.Adam (2e-4, beta_1 = 0.5)

In [ ]:
checkpoint_dir    = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
def generate_images(model, test_input, tar):
    prediction = model(test_input, training=True)
    plt.figure(figsize=(15, 15))

    display_list = [test_input[0], tar[0], prediction[0]]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # Getting the pixel values in the [0, 1] range to plot.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

for example_input, example_target in test_dataset.take(1):
    generate_images(generator, example_input, example_target)

In [ ]:
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit_maps/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def train_step(input_image, target, step):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = generator(input_image, training=True)

        disc_real_output = discriminator([input_image, target], training=True)
        disc_generated_output = discriminator([input_image, gen_output], training=True)

        gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

    with summary_writer.as_default():
        tf.summary.scalar('gen_total_loss', gen_total_loss, step=step//1000)
        tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=step//1000)
        tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=step//1000)
        tf.summary.scalar('disc_loss', disc_loss, step=step//1000)

In [ ]:
def fit(train_ds, test_ds, steps):
    example_input, example_target = next(iter(test_ds.take(1)))
    start = time.time()

    for step, (input_image, target) in train_ds.repeat().take(steps).enumerate():
        if (step) % 1000 == 0:
            display.clear_output(wait=True)

            if step != 0:
                print(f'Time taken for 1000 steps: {time.time()-start:.2f} sec\n')

            start = time.time()

            generate_images(generator, example_input, example_target)
            print(f"Step: {step//1000}k")

        train_step(input_image, target, step)

        # Training step
        if (step+1) % 10 == 0:
            print('.', end='', flush=True)


        # Save (checkpoint) the model every 5k steps
        if (step + 1) % 5000 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}
fit(train_dataset, test_dataset, steps=40000)

In [ ]:
!ls {checkpoint_dir}

In [ ]:
# Run the trained model on a few examples from the test set
for inp, tar in test_dataset.take(10):
    generate_images(generator, inp, tar)

# Utilización imagen ETSE - GoogleMaps

In [ ]:
import numpy as np 

img_etse_c = "etse_c.jpg"
img_etse_s = "etse_s.jpg"
# Abre la imagen en formato JPEG

img_c = tf.io.read_file(img_etse_c)
img_c = tf.io.decode_jpeg(img_c)

img_s = tf.io.read_file(img_etse_s)
img_s = tf.io.decode_jpeg(img_s)

# Cast type to float32
img_c = tf.cast(img_c, tf.float32)
img_s = tf.cast(img_s, tf.float32)

# Muestra la imagen
plt.imshow(img_c/255.)
plt.show()
 
plt.imshow(img_s/255.)
plt.show()

In [ ]:
input_image, real_image = resize(img_c, img_s,
                                   IMG_HEIGHT, IMG_WIDTH)

print
inp, tar = normalize(input_image, real_image)

inp = np.expand_dims(inp, axis=0)
tar = np.expand_dims(tar, axis=0)

plt.imshow(inp[0,:,:,:])
plt.show()

plt.imshow(tar[0,:,:,:])
plt.show()

### **Testeamos la arquitectura pix2pix entrenada con el dataset maps con el mapa de la ETSE esxtraido de googlemaps**

In [ ]:
generate_images(generator, inp, tar)

### Ejercicio prelab:
Corre el código de cyclegan de la página de tensorflow:
https://www.tensorflow.org/tutorials/generative/cyclegan

### · Ejercicio 2:
Modifica el código de cyclegan de la página de tensorflow (el anterior) para que funcione sobre el dataset de documentos sucios que os proporcionamos. Comprueba que los modelos han aprendido a "limpiar" y a "ensuciar" documentos.


Para modificar el código de CycleGAN de TensorFlow para funcionar en el conjunto de datos de documentos sucios proporcionado, necesitamos ajustar los siguientes parámetros:

- BUFFER_SIZE: la cantidad de elementos a mezclar en el conjunto de datos.

- BATCH_SIZE: el tamaño del lote para el entrenamiento.

- IMG_WIDTH: el ancho de las imágenes en el conjunto de datos.

- IMG_HEIGHT: la altura de las imágenes en el conjunto de datos.

- OUTPUT_CHANNELS: el número de canales de salida para las imágenes generadas.

- dataset: la ubicación del conjunto de datos de documentos sucios.

- gan_model: el modelo GAN que se utilizará.

Además, también necesitamos ajustar las funciones de carga de imagen para que carguen y preprocesen imágenes del conjunto de datos de documentos sucios.


## Original

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

AUTOTUNE = tf.data.AUTOTUNE

### DATASETS

In [ ]:
dataset, metadata = tfds.load('cycle_gan/horse2zebra',
                              with_info=True, as_supervised=True)

train_horses, train_zebras = dataset['trainA'], dataset['trainB']
test_horses, test_zebras   = dataset['testA'], dataset['testB']

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE  = 1
IMG_WIDTH   = 256
IMG_HEIGHT  = 256

In [ ]:
def random_crop(image):
    cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image

# normalizing the images to [-1, 1]
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = random_crop(image)

    # random mirroring
    image = tf.image.random_flip_left_right(image)

    return image

def preprocess_image_train(image, label):
    image = random_jitter(image)
    image = normalize(image)
    return image

def preprocess_image_test(image, label):
    image = normalize(image)
    return image

train_horses = train_horses.cache().map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

train_zebras = train_zebras.cache().map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

test_horses = test_horses.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

test_zebras = test_zebras.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

###  pix2pix

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator (OUTPUT_CHANNELS, norm_type = 'instancenorm')
generator_f = pix2pix.unet_generator (OUTPUT_CHANNELS, norm_type = 'instancenorm')

discriminator_x = pix2pix.discriminator (norm_type = 'instancenorm', target=False)
discriminator_y = pix2pix.discriminator (norm_type = 'instancenorm', target=False)

sample_horse = next(iter(train_horses))
sample_zebra = next(iter(train_zebras))

to_zebra = generator_g(sample_horse)
to_horse = generator_f(sample_zebra)
plt.figure(figsize=(8, 8))
contrast = 8

imgs = [sample_horse, to_zebra, sample_zebra, to_horse]
title = ['Horse', 'To Zebra', 'Zebra', 'To Horse']

for i in range(len(imgs)):
  plt.subplot(2, 2, i+1)
  plt.title(title[i])
  if i % 2 == 0:
    plt.imshow(imgs[i][0] * 0.5 + 0.5)
  else:
    plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)
plt.show()

plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Is a real zebra?')
plt.imshow(discriminator_y(sample_zebra)[0, ..., -1], cmap='RdBu_r')

plt.subplot(122)
plt.title('Is a real horse?')
plt.imshow(discriminator_x(sample_horse)[0, ..., -1], cmap='RdBu_r')

plt.show()

### LOSS

In [ ]:
LAMBDA = 10

loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return LAMBDA * loss1

def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

### checkpoint

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')
    


### TRAIN

In [ ]:
EPOCHS = 10

def generate_images(model, test_input):
    prediction = model(test_input)

    plt.figure(figsize=(12, 12))

    display_list = [test_input[0], prediction[0]]
    title = ['Input Image', 'Predicted Image']

    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
        plt.show()

@tf.function
def train_step(real_x, real_y):
    # persistent is set to True because the tape is used more than
    # once to calculate the gradients.
    with tf.GradientTape(persistent=True) as tape:
        # Generator G translates X -> Y
        # Generator F translates Y -> X.

        fake_y = generator_g(real_x, training=True)
        cycled_x = generator_f(fake_y, training=True)

        fake_x = generator_f(real_y, training=True)
        cycled_y = generator_g(fake_x, training=True)

        # same_x and same_y are used for identity loss.
        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_y, training=True)

        disc_real_x = discriminator_x(real_x, training=True)
        disc_real_y = discriminator_y(real_y, training=True)

        disc_fake_x = discriminator_x(fake_x, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)

        # calculate the loss
        gen_g_loss = generator_loss(disc_fake_y)
        gen_f_loss = generator_loss(disc_fake_x)

        total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)

        # Total generator loss = adversarial loss + cycle loss
        total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
        total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
    # Calculate the gradients for generator and discriminator
    generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)

    discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)

    # Apply the gradients to the optimizer
    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))

    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))

    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

for epoch in range(EPOCHS):
    start = time.time()

    n = 0
    for image_x, image_y in tf.data.Dataset.zip((train_horses, train_zebras)):
        train_step(image_x, image_y)
        if n % 10 == 0:
            print ('.', end='')
        n += 1

    clear_output(wait=True)
        # Using a consistent image (sample_horse) so that the progress of the model
        # is clearly visible.
    generate_images(generator_g, sample_horse)

    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                                 ckpt_save_path))

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                              time.time()-start))

## Run the trained model on the test dataset
for inp in test_horses.take(5):
    generate_images(generator_g, inp)

Si nod fijamos bien, vemos algunos fallso de transformación. Esto puede ser debido al colapso del modelo en el entrenamiento o que existen situaciones expeciales que no se han tomado en cuenta en el entrenamiento. 

## Modificación

El modelo CycleGAN también utiliza la arquitectura U-Net que espera entradas de imágenes de tamaño 256x256x3, por lo que es necesario realizar algunas adaptaciones para que funcione correctamente con imágenes de 258x540x1. 

Una posible solución es redimensionar las imágenes de entrada a un tamaño compatible con la arquitectura del modelo. En este caso redimensionaremos a 256x512x

### DATA

In [ ]:
import tensorflow as tf

import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow_examples.models.pix2pix import pix2pix

from tensorflow.keras.preprocessing.image import load_img, img_to_array

import os
import time
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

AUTOTUNE = tf.data.AUTOTUNE

# Define las rutas a los directorios de train y train_dirty
PATH = 'docs'
input_dirted  = 'docs/train'
input_cleaned = 'docs/train_cleaned'

# cargar y preprocesar los datasets de imágenes
train_dirted  = tf.data.Dataset.list_files (str (input_dirted +'/*.png'))
train_cleaned = tf.data.Dataset.list_files (str (input_cleaned +'/*.png'))

for element in train_dirted:
    print (element)
    
# Ajuste de parámetros para el conjunto de datos de documentos sucios
BUFFER_SIZE     = 13
BATCH_SIZE      = 1
IMG_WIDTH       = 258
IMG_HEIGHT      = 540
M_IMG_WIDTH       = 256
M_IMG_HEIGHT      = 512 
OUTPUT_CHANNELS = 3
IMG_CHANNELS    = 1

R_IMG_WIDTH       = 288
R_IMG_HEIGHT      = 570

In [ ]:
def load(image_file):
    
    # Read and decode an image file to a uint8 tensor
    image = tf.io.read_file (image_file)
    image = tf.io.decode_png (image,3)
    #image = tf.repeat(image, 3, axis=-1) # Replicar el único canal en tres canales
    image = tf.cast(image, tf.float32) / 255.0
 
    return image

def random_crop (image):
    cropped_image = tf.image.random_crop (
                          image, size = [M_IMG_WIDTH, M_IMG_HEIGHT,  3])

    return cropped_image

# normalizing the images to [-1, 1]
def normalize (image):
    ind = tf.cast (image, tf.float32)
    if np.max(ind) <= 1.0 and np.min(ind) >= -1.0:
        print("La imagen está dentro del rango [-1, 1]")
    else:
        print("La imagen no está dentro del rango [-1, 1]")
        ind = (ind / 127.5) - 1
    
    return ind

@tf.function
def random_jitter (image):
    # resizing to 286 x 286 x 3
    ind = tf.image.resize (image, [R_IMG_WIDTH, R_IMG_HEIGHT],
                          method = tf.image.ResizeMethod.NEAREST_NEIGHBOR)
 
    # randomly cropping to 258 x 540 x 1
    ind = random_crop (ind)
 
    # random mirroring
    ind = tf.image.random_flip_left_right (ind)
 
    return ind


def preprocess_image_train (input_image):
    ind = load (input_image)
    ind = random_jitter (ind)
    # ind = normalize (ind) # Nohace falta, las imagenes ya están normalizadas entre {-1,1]
    
    return ind

inp = load(str('docs/train/11.png'))

print (inp.shape)


# Casting to int for matplotlib to display the images
plt.figure()
plt.imshow(inp)
plt.show ()

inp2 = (normalize ( random_crop (load('docs/train/11.png'))) ) 
print (inp2.shape)
plt.imshow ( inp2  )
plt.show ()


inp2 = (normalize ( random_crop (load('docs/train/11.png'))) ) 
print (inp2.shape)
plt.imshow ( inp2  )
plt.show ()



**Vemos que la iamgenes ya estan normalizadas entre [-1,1] por lo que no tenemos que normalizar.**

In [ ]:
train_dirted = train_dirted.cache().map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

train_cleaned = train_cleaned.cache().map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE) 

In [ ]:
sample_dirted  = next (iter (train_dirted))
sample_cleaned = next (iter (train_cleaned))

print (sample_dirted.shape)
plt.imshow ( sample_dirted [0]   )
plt.show ()


print (sample_dirted.shape)
plt.imshow ( sample_cleaned [0] )
plt.show ()

### PIX2PIX

In [ ]:
generator_g = pix2pix.unet_generator (OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator (OUTPUT_CHANNELS, norm_type='instancenorm')
 
discriminator_x = pix2pix.discriminator ( norm_type = 'instancenorm', target = False)
discriminator_y = pix2pix.discriminator ( norm_type = 'instancenorm', target = False)


 

to_dirted  = generator_g (sample_dirted)
to_cleaned = generator_f (sample_cleaned)

plt.figure (figsize = (8, 8))
contrast = 9

imgs  = [ sample_dirted   , to_cleaned,  sample_cleaned   * 127.5 , to_dirted]
title = ['Dirted', 'To Cleaned', 'Cleaned', 'To Dirted']

for i in range (len (imgs)):
    plt.subplot (2, 2, i+1)
    plt.title (title [i])
 
    if i % 2 == 0:
        plt.imshow (imgs [i][0] * 0.5 + 0.5)
    else:
        plt.imshow (imgs [i][0] * 0.5 * contrast + 0.5)
    
plt.show ()

plt.figure (figsize = (8, 8))

plt.subplot (121)
plt.title ('Is a real Cleaned?')
plt.imshow (discriminator_y (sample_cleaned) [0, ..., -1], cmap = 'RdBu_r')

plt.subplot (122)
plt.title ('Is a real Dirted?')
plt.imshow (discriminator_x (sample_dirted) [0, ..., -1], cmap = 'RdBu_r')

plt.show ()
   

### LOSS

In [ ]:
LAMBDA = 10

loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return LAMBDA * loss1

def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

### checkpoint

In [ ]:
checkpoint_path = "./checkpoints/train_docs"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

### Train

In [ ]:
EPOCHS = 40

def generate_images (model, test_input):
    prediction = model (test_input)
    print (prediction.shape)
    plt.figure (figsize = (12,12))
     
    #test_input  = (test_input + 1)* 127.5
    #prediction  = (prediction + 1)* 127.5
    
    display_list = [test_input [0], prediction [0]]
    title = ['Input Image', 'Predicted Image']

    for i in range (2):
        plt.subplot (1, 2, i+1)
        plt.title (title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow (display_list[i]*0.5+0.5)
        plt.axis ('off')
        plt.show ()

@tf.function
def train_step(real_x, real_y):
    # persistent is set to True because the tape is used more than
    # once to calculate the gradients.
    with tf.GradientTape(persistent=True) as tape:
        # Generator G translates X -> Y
        # Generator F translates Y -> X.

        fake_y   = generator_g(real_x, training=True)
        cycled_x = generator_f(fake_y, training=True)

        fake_x   = generator_f(real_y, training=True)
        cycled_y = generator_g(fake_x, training=True)

        # same_x and same_y are used for identity loss.
        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_y, training=True)

        disc_real_x = discriminator_x(real_x, training=True)
        disc_real_y = discriminator_y(real_y, training=True)

        disc_fake_x = discriminator_x(fake_x, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)

        # calculate the loss
        gen_g_loss = generator_loss(disc_fake_y)
        gen_f_loss = generator_loss(disc_fake_x)

        total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)

        # Total generator loss = adversarial loss + cycle loss
        total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
        total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
    # Calculate the gradients for generator and discriminator
    generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)

    discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)

    # Apply the gradients to the optimizer
    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))

    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))

    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

for epoch in range(EPOCHS):
    start = time.time()

    n = 0
    for image_x, image_y in tf.data.Dataset.zip((train_dirted, train_cleaned)):
   
        train_step (image_x, image_y)
        if n % 10 == 0:
            print ('.', end='')
        n += 1

    clear_output (wait=True)
        # Using a consistent image (sample_dirted) so that the progress of the model
        # is clearly visible.
    print(' (*) Limpiando documentos:')
    generate_images(generator_g, sample_dirted)
    #print(' (*) Ensuciando documentos:')
    #generate_images(generator_f, sample_cleaned)
    
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                                 ckpt_save_path))

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                              time.time()-start))

**Vemos como la imagen predicha es una documento limpio.**

### Test : Dirted --> cleaned

In [ ]:
for inp in train_dirted.take(5):
    generate_images(generator_g, inp  )

### Test : cleaned  --> Dirted

In [ ]:
for inp in train_cleaned.take(5):
    generate_images(generator_f, inp  )

Comprobmo que los documentos sucios los limpia con generator_g. Y los documentos limpios lso ensucia con genertor_f